In [1]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Carregar os dados
dataset = load_dataset('csv', data_files={
                                          'train': '/content/drive/MyDrive/Colab Notebooks/df_stj_train_zsl.csv',
                                          'validation': '/content/drive/MyDrive/Colab Notebooks/df_stj_validacao_zsl.csv',
                                          'test': '/content/drive/MyDrive/Colab Notebooks/df_stj_teste.csv'
                                          })

# Tokenizador e modelo BERTimbau
tokenizer = BertTokenizerFast.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=7)

Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Label map para o treinamento (6 classes)
label_map_train = {'DIREITO ADMINISTRATIVO': 0,
                   'DIREITO CIVIL': 1,
                   'DIREITO DA CRIANÇA E DO ADOLESCENTE': 2,
                   'DIREITO DO CONSUMIDOR': 3,
                   'DIREITO EMPRESARIAL': 4,
                   'DIREITO PENAL': 5
}

# Label map para o validação (6 classes)
label_map_validation = {'DIREITO ADMINISTRATIVO': 0,
                        'DIREITO CIVIL': 1,
                        'DIREITO DA CRIANÇA E DO ADOLESCENTE': 2,
                        'DIREITO DO CONSUMIDOR': 3,
                        'DIREITO EMPRESARIAL': 4,
                        'DIREITO PENAL': 5
}

# Label map para o teste (7 classes, incluindo a classe não vista)
label_map_test = {'DIREITO ADMINISTRATIVO': 0,
                  'DIREITO CIVIL': 1,
                  'DIREITO DA CRIANÇA E DO ADOLESCENTE': 2,
                  'DIREITO DO CONSUMIDOR': 3,
                  'DIREITO EMPRESARIAL': 4,
                  'DIREITO PENAL': 5,
                  'DIREITO AMBIENTAL': 6
}

def preprocess_training_data(examples):
    examples["label"] = [label_map_train[label] for label in examples["MATERIA"]]
    return tokenizer(examples["EMENTA"], padding="max_length", truncation=True, max_length=512)

def preprocess_validation_data(examples):
    examples["label"] = [label_map_validation[label] for label in examples["MATERIA"]]
    return tokenizer(examples["EMENTA"], padding="max_length", truncation=True, max_length=512)

def preprocess_testing_data(examples):
    examples["label"] = [label_map_test[label] for label in examples["MATERIA"]]
    return tokenizer(examples["EMENTA"], padding="max_length", truncation=True, max_length=512)

encoded_dataset = {}

# Pré-processando o conjunto de treinamento
encoded_dataset["train"] = dataset["train"].map(preprocess_training_data, batched=True)

# Pré-processando o conjunto de validação
encoded_dataset["validation"] = dataset["validation"].map(preprocess_validation_data, batched=True)

# Pré-processando o conjunto de teste
encoded_dataset["test"] = dataset["test"].map(preprocess_testing_data, batched=True)

Map:   0%|          | 0/5140 [00:00<?, ? examples/s]

Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

Map:   0%|          | 0/1111 [00:00<?, ? examples/s]

In [6]:
# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
)

# Função de métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    precision = precision_score(labels, preds, average="weighted", zero_division=0)
    recall = recall_score(labels, preds, average="weighted", zero_division=0)

     # Calcular a acurácia harmônica
    acc_seen = accuracy_score(labels[labels != -100], preds[labels != -100])  # Acurácia das classes vistas
    acc_unseen = accuracy_score(labels[labels == -100], preds[labels == -100])  # Acurácia das classes não vistas
    harmonic_mean = 2 * (acc_seen * acc_unseen) / (acc_seen + acc_unseen) if (acc_seen + acc_unseen) > 0 else 0

    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall, "harmonic_mean": harmonic_mean}

In [7]:
# Configuração do treinador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics
)

# Treinar o modelo
trainer.train()

# Avaliar o modelo
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']}")
print(f"F1 Score: {eval_results['eval_f1']}")
print(f"Precision: {eval_results['eval_precision']}")
print(f"Recall: {eval_results['eval_recall']}")
print(f"Acurácia Harmônica: {eval_results['eval_harmonic_mean']}")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Harmonic Mean
1,No log,0.384649,0.884477,0.881890,0.881759,0.884477,0
2,0.281500,0.358727,0.888989,0.886141,0.885539,0.888989,0
3,0.281500,0.351115,0.896209,0.894662,0.894250,0.896209,0


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Accuracy: 0.8962093862815884
F1 Score: 0.8946621158775053
Precision: 0.8942502086472234
Recall: 0.8962093862815884
Acurácia Harmônica: 0


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
